In [ ]:
# Standard library imports
import os
import time

# Local imports
from utils.general import printt, print_sucess, N_PROCESSES
from utils.data_load_extract import (
    ensure_directory_exists_and_is_empty,
    extract_sources_from_suspicious_xml
    )
from utils.preprocessing import preprocessing_data
from utils.filtering import document_filtering, sentence_filtering
from utils.metrics import similarity_computation, evaluate_detection

In [ ]:
reload_sources = True
reload_suspicious = True
reload_sent_matches = True
results_filename = "data_for_figure_20.txt"
SOURCE_LIMIT = 64
SUSPICIOUS_LIMIT = 8
THRESHOLD_LIST = [0.65] # [0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70]
BETA = 0.5

In [ ]:
# 1. Initial setup (paths and number of processes for parallelization)
printt("Starting the algorithm...")
# Getting the current directory path and setting up the base path for data.
current_dir = os.getcwd()
base_data_path = os.path.join(current_dir, "pan-plagiarism-corpus-2011/external-detection-corpus")
sent_matches_dir = os.path.join(current_dir, "sent-matches-dir")
# Print the number of processes that will be used for parallel operations.
printt(f"N_PROCESSES: {N_PROCESSES}")

In [ ]:
# 2. Directory setup
printt("Setting up directories...")
# Initialize the directories for raw and cleaned versions of both source and suspicious documents.
# Set up paths for raw and cleaned source documents.
cleaned_source_path = os.path.join(current_dir, "cleaned-source-documents")
source_path = os.path.join(base_data_path, "source-document")
# Set up paths for raw and cleaned suspicious documents.
cleaned_suspicious_path = os.path.join(current_dir, "cleaned-suspicious-documents")
suspicious_path = os.path.join(base_data_path, "suspicious-document")
if reload_sources:
    ensure_directory_exists_and_is_empty(cleaned_source_path)
if reload_suspicious:
    ensure_directory_exists_and_is_empty(cleaned_suspicious_path)
if reload_sent_matches:
    ensure_directory_exists_and_is_empty(sent_matches_dir)

In [ ]:
# 3. Extract real results from XML files to understand the ground truth of plagiarism references
printt("Extracting reference results...")
# Extract and store the plagiarism references from the suspicious XML files
df_plagiarism_references = extract_sources_from_suspicious_xml(suspicious_path)

In [ ]:
df_plagiarism_references.head()

In [ ]:
# Step 4: Preprocess the raw data to prepare it for the subsequent analysis
printt("Preprocessing data...")
printt("Source documents preprocessing...")
if reload_sources:
    preprocessing_data(source_path, cleaned_source_path, SOURCE_LIMIT)
printt("Suspicious documents preprocessing...")
if reload_suspicious:
    preprocessing_data(suspicious_path, cleaned_suspicious_path, SUSPICIOUS_LIMIT)

In [ ]:
# Step 5: Identify potential source documents for each suspicious document
printt("Filtering documents...")
suspicious_to_sources = document_filtering(cleaned_suspicious_path, cleaned_source_path)

In [ ]:
# Step 6: Filter sentences based on potential plagiarisms
printt("Filtering sentences...")
if reload_sent_matches:
    sentence_filtering(cleaned_suspicious_path, cleaned_source_path, suspicious_to_sources, sent_matches_dir)

In [ ]:
# Step 7: Compute and save similarity scores between suspicious and source sentences
printt("Processing similarity calculations...")
output_dir = os.path.join(current_dir, "sentences-detections")
df_similarity = similarity_computation(cleaned_suspicious_path, cleaned_source_path, sent_matches_dir, BETA)


In [ ]:
# TODO: refactor step 7 and 8 to work better with step 6
# Step 7 reloaded: compute similarity scores for each of the parque files in sent_matches_dir


In [ ]:

df_similarity.to_parquet("df_similarity.parquet")
for THRESHOLD in THRESHOLD_LIST:
    printt(
        f"SOURCE_LIMIT: {SOURCE_LIMIT}, SUSPICIOUS_LIMIT: {SUSPICIOUS_LIMIT}, THRESHOLD: {THRESHOLD}, BETA: {BETA}"
    )
    df_plagiarism_detections = df_similarity[
        df_similarity["hybrid_similarity"] > THRESHOLD
    ]
    # 8. Analysis of detections
    printt("Evaluate detections...")
    precision, recall, f1_score, plagdet = evaluate_detection(
        df_plagiarism_references, df_plagiarism_detections
    )
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")
    print(f"Plagdet-Score: {plagdet:.4f}")
    # 9. Save results
    printt(f"Saving results to {results_filename}")
    with open(results_filename, "a") as f:
        f.write(
            f"{SUSPICIOUS_LIMIT}, {THRESHOLD}, {BETA}, {precision}, {recall}, {f1_score}, {plagdet}\n"
        )

In [ ]:
if __name__ == "__main__":
    # Replication of Figure 20
    print("Replication of Figure 20")
    start_time = time.time()

    reload_sources = True
    reload_suspicious = True

    results_filename = "data_for_figure_20.txt"
    SOURCE_LIMIT = 64
    SUSPICIOUS_LIMIT = 8
    THRESHOLD_LIST = [0.65] # [0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70]
    BETA = 0.5

    # 1. Initial setup (paths and number of processes for parallelization)
    printt("Starting the algorithm...")

    # Getting the current directory path and setting up the base path for data.
    current_dir = os.getcwd()
    base_data_path = os.path.join(
        current_dir, "pan-plagiarism-corpus-2011/external-detection-corpus"
    )

    # Print the number of processes that will be used for parallel operations.
    printt("N_PROCESSES:", N_PROCESSES)

    # 2. Directory setup
    printt("Setting up directories...")

    # Initialize the directories for raw and cleaned versions of both source and suspicious documents.
    # Set up paths for raw and cleaned source documents.
    cleaned_source_path = os.path.join(current_dir, "cleaned-source-documents")
    source_path = os.path.join(base_data_path, "source-document")

    # Set up paths for raw and cleaned suspicious documents.
    cleaned_suspicious_path = os.path.join(current_dir, "cleaned-suspicious-documents")
    suspicious_path = os.path.join(base_data_path, "suspicious-document")

    if reload_sources:
        ensure_directory_exists_and_is_empty(cleaned_source_path)
    if reload_suspicious:
        ensure_directory_exists_and_is_empty(cleaned_suspicious_path)

    # 3. Extract real results from XML files to understand the ground truth of plagiarism references
    printt("Extracting reference results...")

    # Extract and store the plagiarism references from the suspicious XML files
    df_plagiarism_references = extract_sources_from_suspicious_xml(suspicious_path)

    # Step 4: Preprocess the raw data to prepare it for the subsequent analysis
    printt("Preprocessing data...")
    printt("Source documents preprocessing...")
    if reload_sources:
        preprocessing_data(source_path, cleaned_source_path, SOURCE_LIMIT)

    printt("Suspicious documents preprocessing...")
    if reload_suspicious:
        preprocessing_data(suspicious_path, cleaned_suspicious_path, SUSPICIOUS_LIMIT)

    # Step 5: Identify potential source documents for each suspicious document
    printt("Filtering documents...")
    suspicious_to_sources = document_filtering(cleaned_suspicious_path, cleaned_source_path)

    # Step 6: Filter sentences based on potential plagiarisms
    printt("Filtering sentences...")
    df_sentences_detections = sentence_filtering(cleaned_suspicious_path, cleaned_source_path, suspicious_to_sources)
    df_sentences_detections.to_parquet("df_sentences_detections.parquet")

    printt("Divide Dataframe Sentences...")
    divide_df_sentences(current_dir)

    # Step 7: Compute and save similarity scores between suspicious and source sentences
    printt("Processing similarity calculations...")
    output_dir = os.path.join(current_dir, "sentences-detections")
    df_similarity = similarity_computation(
        cleaned_suspicious_path, cleaned_source_path, output_dir, BETA
    )

    df_similarity.to_parquet("df_similarity.parquet")

    for THRESHOLD in THRESHOLD_LIST:
        printt(
            f"SOURCE_LIMIT: {SOURCE_LIMIT}, SUSPICIOUS_LIMIT: {SUSPICIOUS_LIMIT}, THRESHOLD: {THRESHOLD}, BETA: {BETA}"
        )

        df_plagiarism_detections = df_similarity[
            df_similarity["hybrid_similarity"] > THRESHOLD
        ]

        # 8. Analysis of detections
        printt("Evaluate detections...")
        precision, recall, f1_score, plagdet = evaluate_detection(
            df_plagiarism_references, df_plagiarism_detections
        )
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1_score:.4f}")
        print(f"Plagdet-Score: {plagdet:.4f}")

        # 9. Save results
        printt(f"Saving results to {results_filename}")
        with open(results_filename, "a") as f:
            f.write(
                f"{SUSPICIOUS_LIMIT}, {THRESHOLD}, {BETA}, {precision}, {recall}, {f1_score}, {plagdet}\n"
            )

    end_time = time.time()
    diff_time = end_time - start_time
    print(f"Time taken for Figure 20 replication: {diff_time:.2f} seconds")

    print_sucess()


In [ ]:
    # Replication of Figure 20
    print("Replication of Figure 20")
    start_time = time.time()

    reload_sources = True
    reload_suspicious = True

    results_filename = "data_for_figure_20.txt"
    SOURCE_LIMIT = 64
    SUSPICIOUS_LIMIT = 8
    THRESHOLD_LIST = [0.65] # [0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70]
    BETA = 0.5

    # 1. Initial setup (paths and number of processes for parallelization)
    printt("Starting the algorithm...")

    # Getting the current directory path and setting up the base path for data.
    current_dir = os.getcwd()
    base_data_path = os.path.join(
        current_dir, "pan-plagiarism-corpus-2011/external-detection-corpus"
    )

    # Print the number of processes that will be used for parallel operations.
    printt("N_PROCESSES:", N_PROCESSES)

    # 2. Directory setup
    printt("Setting up directories...")

    # Initialize the directories for raw and cleaned versions of both source and suspicious documents.
    # Set up paths for raw and cleaned source documents.
    cleaned_source_path = os.path.join(current_dir, "cleaned-source-documents")
    source_path = os.path.join(base_data_path, "source-document")

    # Set up paths for raw and cleaned suspicious documents.
    cleaned_suspicious_path = os.path.join(current_dir, "cleaned-suspicious-documents")
    suspicious_path = os.path.join(base_data_path, "suspicious-document")

    if reload_sources:
        ensure_directory_exists_and_is_empty(cleaned_source_path)
    if reload_suspicious:
        ensure_directory_exists_and_is_empty(cleaned_suspicious_path)

    # 3. Extract real results from XML files to understand the ground truth of plagiarism references
    printt("Extracting reference results...")

    # Extract and store the plagiarism references from the suspicious XML files
    df_plagiarism_references = extract_sources_from_suspicious_xml(suspicious_path)

    # Step 4: Preprocess the raw data to prepare it for the subsequent analysis
    printt("Preprocessing data...")
    printt("Source documents preprocessing...")
    if reload_sources:
        preprocessing_data(source_path, cleaned_source_path, SOURCE_LIMIT)

    printt("Suspicious documents preprocessing...")
    if reload_suspicious:
        preprocessing_data(suspicious_path, cleaned_suspicious_path, SUSPICIOUS_LIMIT)

    # Step 5: Identify potential source documents for each suspicious document
    printt("Filtering documents...")
    suspicious_to_sources = document_filtering(cleaned_suspicious_path, cleaned_source_path)

    # Step 6: Filter sentences based on potential plagiarisms
    printt("Filtering sentences...")
    df_sentences_detections = sentence_filtering(cleaned_suspicious_path, cleaned_source_path, suspicious_to_sources)
    df_sentences_detections.to_parquet("df_sentences_detections.parquet")

    printt("Divide Dataframe Sentences...")
    divide_df_sentences(current_dir)

    # Step 7: Compute and save similarity scores between suspicious and source sentences
    printt("Processing similarity calculations...")
    output_dir = os.path.join(current_dir, "sentences-detections")
    df_similarity = similarity_computation(
        cleaned_suspicious_path, cleaned_source_path, output_dir, BETA
    )

    df_similarity.to_parquet("df_similarity.parquet")

    for THRESHOLD in THRESHOLD_LIST:
        printt(
            f"SOURCE_LIMIT: {SOURCE_LIMIT}, SUSPICIOUS_LIMIT: {SUSPICIOUS_LIMIT}, THRESHOLD: {THRESHOLD}, BETA: {BETA}"
        )

        df_plagiarism_detections = df_similarity[
            df_similarity["hybrid_similarity"] > THRESHOLD
        ]

        # 8. Analysis of detections
        printt("Evaluate detections...")
        precision, recall, f1_score, plagdet = evaluate_detection(
            df_plagiarism_references, df_plagiarism_detections
        )
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1_score:.4f}")
        print(f"Plagdet-Score: {plagdet:.4f}")

        # 9. Save results
        printt(f"Saving results to {results_filename}")
        with open(results_filename, "a") as f:
            f.write(
                f"{SUSPICIOUS_LIMIT}, {THRESHOLD}, {BETA}, {precision}, {recall}, {f1_score}, {plagdet}\n"
            )

    end_time = time.time()
    diff_time = end_time - start_time
    print(f"Time taken for Figure 20 replication: {diff_time:.2f} seconds")

    print_sucess()